In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2>준비하기</h2>
데이터를 처리하기 전에,<br>
데이터에 필요한 기본 라이브러리를 불러온다.

In [ ]:
#라이브러리 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#데이터 불러오기
dt=pd.read_csv("../input/predict-test-scores-of-students/test_scores.csv")
dt.head()

<h2>EDA</h2>

In [ ]:
#데이터 살펴보기
dt.info()
dt.describe()

In [ ]:
#위의 코드로 살펴본 데이터중 school type관련된 unique 값 살펴보기
dt['school_type'].unique()

학교의 종류(?)는 두개의 병목형 변수로 이루어져 있음을 알 수 있다.

In [ ]:
#위의 코드로 살펴본 데이터중 school관련된 unique 값 살펴보기
dt['school'].unique()

학교의 이름을 나열한 값인것 같다.<br>
관측치로 제시된 값이 많긴 하지만 학교별로 점수가 다를수도 있음을 고려하여<br>
일단 이런값을 가지고 있음을 숙지하고만 있자.

In [ ]:
#위의 코드로 살펴본 데이터중 teaching method관련된 unique 값 살펴보기
dt['teaching_method'].unique()

가르치는 방법은 두가지 방식으로 나뉘어져있다.

In [ ]:
#위의 코드로 살펴본 데이터중 lunch관련된 unique 값 살펴보기
dt['lunch'].unique()

lunch는 뭘 의미하는건지 궁금해서 살펴봤는데(개인적으로 급식/ 도시락인줄 알았음)<br>
이게 뭘 의미하는건지는 잘 모르겠다...<br>
하지만 두가지 병목형 변수로 나뉘어져있다.

<h3>데이터 그래프로 살펴보기</h3>
데이터를 그래프로 살펴보기로 하였다.<br>
그래프를 보는데 제목에서 한글로 쓰면 깨짐현상이 일어나서 다음과 같은 코드를 추가하였다<br>
근데 계속 차트의 제목에서 깨짐현상이 일어나서 <br>
그냥 제목을 영어로 수정하였다.

In [ ]:
import platform
import matplotlib.font_manager as fm

#matplotlib 패키지 한글 깨짐 처리 시작
#------------------------------------------------------------------------------------
# 운영체제별 한글 폰트 설정

if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
    
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

#------------------------------------------------------------------------------------
#matplotlib 패키지 한글 깨짐 처리 끝

#windows
plt.rcParams['font.family'] = "NanumBarunGothic"
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
x = dt.posttest.values

sns.kdeplot(x)
plt.title("posttest data distribution")
plt.show()

posttest는 다음과 같이 80점에 분포가 제일 많은것으로 보인다.

In [ ]:
y = dt.pretest.values

sns.kdeplot(y)
plt.title("pretest data distribution")
plt.show()

pretest는 60점에 제일 분포가 제일 많은것으로 보인다.<br>
posttest가 후의 점수이므로 전반적으로 20점이 상승한것으로 보임다.

In [ ]:
sns.countplot(x="school_type", data=dt)
plt.title("school type data distribution")
plt.show()

scool type을 변수별로 센 그래프다.<br>
하지만 이 그래프로는 school type이 점수에 어떤 영향을 미치는지 잘 모르겠어서<br>
밑에 다음과 같은 그래프를 그렸다.

In [ ]:
sns.violinplot(x="school_type",y="posttest", data=dt)
plt.title("school type data distribution")
plt.show()

위의 그래프를 보면<br> 
non-public의 분포는 80점이 제일 많고, public의 분포는 60-70점이 제일 많다.<br>
평균적으로 non-pubilc의 점수가 public의 점수보다 더 높은것으로 알 수 있다.

In [ ]:
chart=sns.countplot(x="school", data=dt)
plt.title("school data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
sns.violinplot(x="school",y="posttest", data=dt)
plt.title("school data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

맨 위의 그래프는 school의 숫자를 종류별로 세어놓은 그래프다.<br>
그러나 scool의 숫자를 세는것은 학교별 점수의 특징을 파악하는데 별 도움이 되지 않는다고 생각했기 때문에<br>
violinpilot을 이용하여 다시 그래프를 그려보았다.<br>
그 결과, 특정 학교는 점수가 다른 학교보다 높은걸 볼 수 있었다.<br>
반대로 어떤 학교는 다른 학교보다 점수의 분포가 전반적으로 낮은걸 볼 수 있었다<br>
이게 점수에 어떤 영향을 미치는지는 뒤에서 분석을 해야 알 수 있다고 판단하였다.

In [ ]:
chart=sns.countplot(x="classroom", data=dt)
plt.title("school data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

반별로 값을 세어놓은 그래프이다.<br>
이것도 violinplot으로 살펴볼까 생각했지만 label이 너무 세분화 되어있어서<br>
그래프를 그리는것이 별 도움이 되지 않을것이라고 판단하였다.

In [ ]:
sns.countplot(x="teaching_method", data=dt)
plt.title("teaching method data distribution")
plt.show()

가르치는 방법별로 그 값을 세어놓은 그래프다.<br>
이값은 학교별로 다를것이고, 따라서 이렇게 값을 세는건 성적과의 연관을 알아내는것에 별 도움이 되지 않을것이라고 생각 했기 때문에<br>
violinplot을 그려서 그 차이를 알아보았다.

In [ ]:
sns.violinplot(x="teaching_method",y="posttest", data=dt)
plt.title("classroom data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

다음과 같이 경험 위주로 학습을 한 학생들의 평균이 더 높은것을 볼 수 있다.

In [ ]:
sns.countplot(x="school_setting", data=dt)
plt.title("school setting data distribution")
plt.show()

학교의 위치별로 변수별 그 값을 세어놓은 그래프이다.<br>
이 또한 성적과의 연관을 알기엔 무리가 있다고 판단하여 violinplot으로 그 값을 알아보았다.

In [ ]:
sns.violinplot(x="school_setting",y="posttest", data=dt)
plt.title("classroom data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

urban<rural<suburban 순으로<br>
suburban값이 제일 높은값에 제일 많이 분포되어있다.<br>
아마 개인적인 판단으로 도시는 너무 교육열이 높아서 치열해진 결과 시험의 난이도가 높아져서 전반적으로 낮은 점수가,<br>
시골은 학업에 관심도는 비교적 낮아서 시험의 난이도가 비교적 낮아 점수가 높은것 아닐까라는 개인적 생각이 들었다.

In [ ]:
sns.countplot(x="gender", data=dt)
plt.title("school setting data distribution")
plt.show()

In [ ]:
sns.violinplot(x="gender",y="posttest", data=dt)
plt.title("classroom data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

성별간 성적 분포를 그린 그래프다.<br>
여성과 남성 둘다 비슷한 분포를 보인다.

In [ ]:
sns.violinplot(x="n_student",y="posttest", data=dt)
plt.title("classroom data distribution")
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

반에 있는 학생 수 별 점수 분보를 그린 그래프이다.<br>
그래프가 거의 각 변수별로 양 끝쪽의 그래프를 제외하고 일자모양이다.<br>
이는 거의 고르게 분포가 되어있다는 의미이지만 양 끝값이 영향을 줄 수 도 있기 때문에 일단 지켜보도록 한다.

<h2>Feature Engineering</h2>

In [ ]:
dt.columns.values

In [ ]:
#각 변수별 상관계수 값 구해보기
ext_data=dt[['school', 'school_setting', 'school_type', 'classroom',
       'teaching_method', 'n_student', 'student_id', 'gender', 'lunch',
       'pretest', 'posttest']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
#필요없는 변수 drop
#각자 고유의 값을 가지는것
dt_delete=dt.drop(["student_id","classroom"],axis=1)

#classroom은 뒤의 과정에서 classroom값이 오류로 뜨기 때문에 값을 제거하였다.

In [ ]:
#타입이 object인 값을 읽지 못하므로  one hot encoding으로 변환

fe_dt=pd.get_dummies(dt_delete,columns=['school'])

str_type=['school_setting', 'school_type','teaching_method','n_student', 'gender', 'lunch']
    
for i in str_type:
    fe_dt = pd.get_dummies(fe_dt,columns=[i])

fe_dt.head()

In [ ]:
#재확인
fe_dt.columns.values

밑의 코드는 원래 계획에는 없던 코드다.<br>
posttest값이 일일이 죄다 라벨링이 되어서 train값과 test값이 서로 맞지 않는다는 오류가 나와서<br>
train과 test에서 가지는 posttest의 라벨을 일치시겨야 했다. <br>
따라서 posttest평균을 넘으면 1, 넘지 못하면 0을 값을 변환시켜야 했다.

In [ ]:
fe_dt['result']=[1 if i >=fe_dt['posttest'].mean() else 0 for i in fe_dt['posttest']]

<h2> LGBM</h2>

test값과 train값이 나눠진 대출값과 달리 주어진 학생 점수는 데이터가 1개여서<br>
한개의 데이터를 임의로 train과 test로 나누는 과정이 필요하였다.

In [ ]:
from sklearn.model_selection import train_test_split

x=fe_dt.drop(['posttest','result'], axis=1)
y=fe_dt['result']

#데이터를 무작위로 섞어서 임의로 train과 test로 나누기
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, shuffle=True, random_state=1004)

In [ ]:
#LGBM 단계 전에 했던 평균이 넘으면 1, 넘지 않으면 0 값을 부여하는 라벨링 확인하는 코드.
import numpy as np
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_test)
y_test_encoded = encoder.transform(y_test)
print(encoder.classes_)

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

위의 결과를 보면 점수를 예측하는데 pretest가 제일 높은 관계도를 보인다.<br>
그 다음은 교육 방법, 학교의 위치 순으로 높은 관계도를 보이고 있다.

<h3>결과 저장하기</h3>

In [ ]:
preds = clf.predict_proba(fe_dt.drop(['posttest','result'], axis=1))[:, 1 ]

In [ ]:
fe_dt['result'] = preds
fe_dt[['result']].to_csv('result_of_student_score.csv', index=False)